In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticxxate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}





E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.13-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
!pip install -U tensorboardcolab
from tensorboardcolab import *


Requirement already up-to-date: tensorboardcolab in /usr/local/lib/python3.6/dist-packages (0.0.22)


Using TensorFlow backend.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive


In [0]:
#tensorFlow para entrenar la red
import tensorflow as tf

# numpy :D
import numpy as np

# Imagenes
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

#libreria números pseudo random
import random
random.seed(a=None)


# Graficado
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline



In [0]:
path = 'drive/Colab Notebooks/'
save_file = path + 'checkpoint/train_model.ckpt'
data_file = path + 'Data'
sizeimage = 50
Frames = 100
filtros = 3

Cargar los datos de entrenamiento.

In [0]:
dataTrainGS=np.load(data_file + str(Frames) + "Cara/TrainGS" + str(Frames) + ".npy")
dataTrainSX=np.load(data_file + str(Frames) + "Cara/TrainSX" + str(Frames) + ".npy")
dataTrainSY=np.load(data_file + str(Frames) + "Cara/TrainSY" + str(Frames) + ".npy")
dataTrainKIR=np.load(data_file + str(Frames) + "Cara/TrainKIR" + str(Frames) + ".npy")
dataTrainY=np.load(data_file + str(Frames) + "Cara/TrainY" + str(Frames) + ".npy")

In [0]:
len(dataTrainY)



643

Este de abajo cambia dependiendo de que filtros este utilizando

In [0]:
dataTrainGS,dataTrainSY,dataTrainSX,dataTrainKIR,dataTrainY = shuffle(dataTrainGS,dataTrainSY,dataTrainSX,dataTrainKIR,dataTrainY)


Un pequeño porcentaje para la validación (aunque a estas alturas, se concluyó no daba ninguna información valiosa)

In [0]:

dataTrainGS = dataTrainGS[int(len(dataTrainGS)/20):]
dataTrainGS2 = dataTrainGS[:int(len(dataTrainGS)/20)]
dataTrainSY = dataTrainSY[int(len(dataTrainSY)/20):]
dataTrainSY2 = dataTrainSY[:int(len(dataTrainSY)/20)]
dataTrainSX = dataTrainSX[int(len(dataTrainSX)/20):]
dataTrainSX2 = dataTrainSX[:int(len(dataTrainSX)/20)]
dataTrainY = dataTrainY[int(len(dataTrainY)/20):]
dataTrainY2 = dataTrainY[:int(len(dataTrainY)/20)]
dataTrainKIR = dataTrainKIR[int(len(dataTrainKIR)/20):]
dataTrainKIR2 = dataTrainKIR[:int(len(dataTrainKIR)/20)]

Cargando los datos para hacer las pruebas y probar la robustez del modelo.

Test es el conjunto de pruebas con personas nunca antes vistas.

Test2 es el conjunto de pruebas con personas vistas.

Se cargó por bloques para evitar que el GPU colapse.

In [0]:
dataTestGS=np.load(data_file + str(Frames) + "Cara/TestGS" + str(Frames) + ".npy",allow_pickle=True)
dataTestTruthGS2=np.load(data_file + str(Frames) + "Cara/ValidationTruthGS" + str(Frames) + ".npy",allow_pickle=True)
dataTestDeceitGS2=np.load(data_file + str(Frames) + "Cara/ValidationDeceitGS" + str(Frames) + ".npy",allow_pickle=True)


In [0]:
dataTestSX=np.load(data_file + str(Frames) + "Cara/TestSX" + str(Frames) + ".npy",allow_pickle=True)
dataTestSY=np.load(data_file + str(Frames) + "Cara/TestSY" + str(Frames) + ".npy",allow_pickle=True)

dataTestTruthSX2=np.load(data_file + str(Frames) + "Cara/ValidationTruthSX" + str(Frames) + ".npy",allow_pickle=True)
dataTestTruthSY2=np.load(data_file + str(Frames) + "Cara/ValidationTruthSY" + str(Frames) + ".npy",allow_pickle=True)

dataTestDeceitSX2=np.load(data_file + str(Frames) + "Cara/ValidationDeceitSX" + str(Frames) + ".npy",allow_pickle=True)
dataTestDeceitSY2=np.load(data_file + str(Frames) + "Cara/ValidationDeceitSY" + str(Frames) + ".npy",allow_pickle=True)


In [0]:

dataTestKIR=np.load(data_file + str(Frames) + "Cara/TestKIR" + str(Frames) + ".npy",allow_pickle=True)

dataTestTruthKIR2=np.load(data_file + str(Frames) + "Cara/ValidationTruthKIR" + str(Frames) + ".npy",allow_pickle=True)

dataTestDeceitKIR2=np.load(data_file + str(Frames) + "Cara/ValidationDeceitKIR" + str(Frames) + ".npy",allow_pickle=True)

In [0]:
dataTestY=np.load(data_file + str(Frames) + "Cara/TestY" + str(Frames) + ".npy",allow_pickle=True)
dataTestTruthY2=np.load(data_file + str(Frames) + "Cara/ValidationTruthY" + str(Frames) + ".npy",allow_pickle=True)
dataTestDeceitY2=np.load(data_file + str(Frames) + "Cara/ValidationDeceitY" + str(Frames) + ".npy",allow_pickle=True)


In [0]:
x = tf.placeholder('float')
y = tf.placeholder('float')
n_classes = 2
batch_size = 10
keep_rate = 0.5




In [0]:
def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='VALID')

def maxpool3d(x):
    #                        size of window         movement of window as you slide about
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')



In [0]:
#Función para finalizar la sesión.
def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()
    
    

In [0]:
def convolutional_neural_network(
    keep_rate = 1,
    learn_rate= 1e-3,#antes erta 1618e-9
    n_classes=2,
    batch_size = 200
    ):
    reset_graph()
    

    maxpool1 = [1,2,2,2,1]
    maxpool2 = [1,2,2,2,1]
    maxpool3 = [1,2,3,3,1]
    maxpool4 = [1,2,3,3,1]
    c1 = 4
    c2 = 8
    c3 = 16
    c4 = 32
    cf = 1*c4
    cf1 = 64
    cf2 = 64
    cf3 = 64
    cf4 = 64
    cf5 = 64
    cf6 = 64
    cf7 = 64
    cf8 = 64
    cf9 = 64
    
    xr   = tf.placeholder('float',[None,Frames,sizeimage,sizeimage])
    xg   = tf.placeholder('float',[None,Frames,sizeimage,sizeimage])
    xb   = tf.placeholder('float',[None,Frames,sizeimage,sizeimage])
    
    xgs   = tf.placeholder('float',[None,Frames,sizeimage,sizeimage])
    
    xsobx = tf.placeholder('float',[None,Frames,sizeimage,sizeimage])
    xsoby = tf.placeholder('float',[None,Frames,sizeimage,sizeimage])

    xlap = tf.placeholder('float',[None,Frames,sizeimage,sizeimage])
    xkir = tf.placeholder('float',[None,Frames,sizeimage,sizeimage])
    
    xoptx = tf.placeholder('float',[None,Frames,sizeimage,sizeimage])
    xopty = tf.placeholder('float',[None,Frames,sizeimage,sizeimage])
    
    prob = tf.placeholder(tf.float32)

    y = tf.placeholder('float')
    #n_classes = 2
    

    #                # 9 imagenes x 3 x 3 kernel, 1 canal, 64 caracteristicas.
    weights = {'W_conv1':tf.Variable(tf.random_normal([9,3,3,1,c1])), 
               #       7 imagenes x 8 x 8 kernel, 2 channels, 8 caracteristicas.
               'W_conv2':tf.Variable(tf.random_normal([7,3,3,c1,c2])),
               #       3imagenes x 8 x 8 kernel, 6 channels, 16 caracteristicas.
               'W_conv3':tf.Variable(tf.random_normal([7,3,3,c2,c3])),
               'W_conv4':tf.Variable(tf.random_normal([7,3,3,c3,c4])),#128
               'W_fc':tf.Variable(tf.random_normal([cf,cf])),#384
               'W_fc1':tf.Variable(tf.random_normal([cf,cf1])),
               'W_fc2':tf.Variable(tf.random_normal([cf1,cf2])),
               'W_fc3':tf.Variable(tf.random_normal([cf2,cf3])),
               'W_fc4':tf.Variable(tf.random_normal([cf3,cf4])),
               'W_fc5':tf.Variable(tf.random_normal([cf4,cf5])),
               'W_fc6':tf.Variable(tf.random_normal([cf5,cf6])),
               'W_fc7':tf.Variable(tf.random_normal([cf6,cf7])),
               'W_fc8':tf.Variable(tf.random_normal([cf7,cf8])),
               'W_fc9':tf.Variable(tf.random_normal([cf8,cf9])),
               'out':tf.Variable(tf.random_normal([cf9, n_classes]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([c1])),
              'b_conv2':tf.Variable(tf.random_normal([c2])),
              'b_conv3':tf.Variable(tf.random_normal([c3])),
              'b_conv4':tf.Variable(tf.random_normal([c4])),
              'b_fc':tf.Variable(tf.random_normal([cf])),
              'b_fc1':tf.Variable(tf.random_normal([cf1])),
              'b_fc2':tf.Variable(tf.random_normal([cf2])),
              'b_fc3':tf.Variable(tf.random_normal([cf3])),
              'b_fc4':tf.Variable(tf.random_normal([cf4])),
              'b_fc5':tf.Variable(tf.random_normal([cf5])),
              'b_fc6':tf.Variable(tf.random_normal([cf6])),
              'b_fc7':tf.Variable(tf.random_normal([cf7])),
              'b_fc8':tf.Variable(tf.random_normal([cf8])),
              'b_fc9':tf.Variable(tf.random_normal([cf9])),
              'out':tf.Variable(tf.random_normal([n_classes]))}

    #                             image z   image Y   image x
    xr1   = tf.reshape(xr,   shape=[-1,      Frames,      sizeimage,      sizeimage,      1])
    xg1   = tf.reshape(xg,   shape=[-1,      Frames,      sizeimage,      sizeimage,      1])
    xb1   = tf.reshape(xb,   shape=[-1,      Frames,      sizeimage,      sizeimage,      1])
    
    xgs1   = tf.reshape(xgs,   shape=[-1,      Frames,      sizeimage,      sizeimage,      1])
    
    xsobx1 = tf.reshape(xsobx, shape=[-1,      Frames,      sizeimage,      sizeimage,      1])
    xsoby1 = tf.reshape(xsoby, shape=[-1,      Frames,      sizeimage,      sizeimage,      1])
    
    xlap1 = tf.reshape(xlap, shape=[-1,      Frames,      sizeimage,      sizeimage,      1])
    xkir1 = tf.reshape(xkir, shape=[-1,      Frames,      sizeimage,      sizeimage,      1])
    
    xoptx1 = tf.reshape(xoptx, shape=[-1,      Frames,      sizeimage,      sizeimage,      1])
    xopty1 = tf.reshape(xopty, shape=[-1,      Frames,      sizeimage,      sizeimage,      1])

    print("filtro")
    conv1gs = tf.nn.relu(conv3d(xgs1, weights['W_conv1']) + biases['b_conv1'])
    print(conv1gs)
    conv1gs = tf.nn.max_pool3d(conv1gs, ksize=maxpool1, strides=maxpool1, padding='SAME')
    print(conv1gs)
    conv2gs = tf.nn.relu(conv3d(conv1gs, weights['W_conv2']) + biases['b_conv2'])
    print(conv2gs)
    conv2gs = tf.nn.max_pool3d(conv2gs, ksize=maxpool2, strides=maxpool2, padding='SAME')
    print(conv2gs)
    conv3gs = tf.nn.relu(conv3d(conv2gs, weights['W_conv3']) + biases['b_conv3'])
    print(conv3gs)
    conv3gs = tf.nn.max_pool3d(conv3gs, ksize=maxpool3, strides=maxpool3, padding='VALID')
    print(conv3gs)
    conv4gs = tf.nn.relu(conv3d(conv3gs, weights['W_conv4']) + biases['b_conv4'])
    print(conv4gs)
    
    print("filtro")
    conv1sobx = tf.nn.relu(conv3d(xsobx1, weights['W_conv1']) + biases['b_conv1'])
    print(conv1sobx)
    conv1sobx = tf.nn.max_pool3d(conv1sobx, ksize=maxpool1, strides=maxpool1, padding='SAME')
    print(conv1sobx)
    conv2sobx = tf.nn.relu(conv3d(conv1sobx, weights['W_conv2']) + biases['b_conv2'])
    print(conv2sobx)
    conv2sobx = tf.nn.max_pool3d(conv2sobx, ksize=maxpool2, strides=maxpool2, padding='SAME')
    print(conv2sobx)
    conv3sobx = tf.nn.relu(conv3d(conv2sobx, weights['W_conv3']) + biases['b_conv3'])
    print(conv3sobx)
    conv3sobx = tf.nn.max_pool3d(conv3sobx, ksize=maxpool3, strides=maxpool3, padding='VALID')
    print(conv3sobx)
    conv4sobx = tf.nn.relu(conv3d(conv3sobx, weights['W_conv4']) + biases['b_conv4'])
    print(conv4sobx)

    print("filtro")
    conv1soby = tf.nn.relu(conv3d(xsoby1, weights['W_conv1']) + biases['b_conv1'])
    print(conv1soby)
    conv1soby = tf.nn.max_pool3d(conv1soby, ksize=maxpool1, strides=maxpool1, padding='SAME')
    print(conv1soby)
    conv2soby = tf.nn.relu(conv3d(conv1soby, weights['W_conv2']) + biases['b_conv2'])
    print(conv2soby)
    conv2soby = tf.nn.max_pool3d(conv2soby, ksize=maxpool2, strides=maxpool2, padding='SAME')
    print(conv2soby)
    conv3soby = tf.nn.relu(conv3d(conv2soby, weights['W_conv3']) + biases['b_conv3'])
    print(conv3soby)
    conv3soby = tf.nn.max_pool3d(conv3soby, ksize=maxpool3, strides=maxpool3, padding='VALID')
    print(conv3soby)
    conv4soby = tf.nn.relu(conv3d(conv3soby, weights['W_conv4']) + biases['b_conv4'])
    print(conv4soby)
    
    print("concatenacion")
    conv = tf.concat([conv4gs, conv4sobx, conv4soby], -1)
    #conv = tf.multiply(conv4gs, conv4kir)
    #conv2 = tf.multiply(conv4sobx, conv4soby)
    #conv = tf.multiply(conv, conv4soby)
    print(conv)
    
    fc = tf.reshape(conv,[-1, cf])
    fc = tf.nn.tanh(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, rate=1-prob)
    print(fc)
    
    fc1 = tf.nn.relu(tf.matmul(fc, weights['W_fc1'])+biases['b_fc1'])
    fc1 = tf.nn.dropout(fc1, rate=1-prob)
    print(fc1)
    
    fc2 = tf.nn.relu(tf.matmul(fc1, weights['W_fc2'])+biases['b_fc2'])
    fc2 = tf.nn.dropout(fc2, rate=1-prob)
    print(fc2)
    
    fc3 = tf.nn.relu(tf.matmul(fc2, weights['W_fc3'])+biases['b_fc3'])
    fc3 = tf.nn.dropout(fc3, rate=1-prob)
    print(fc3)
    
    fc4 = tf.nn.relu(tf.matmul(fc3, weights['W_fc4'])+biases['b_fc4'])
    fc4 = tf.nn.dropout(fc4, rate=1-prob)
    print(fc4)
    
    fc5 = tf.nn.relu(tf.matmul(fc4, weights['W_fc5'])+biases['b_fc5'])
    fc5 = tf.nn.dropout(fc5, rate=1-prob)
    print(fc5)
    
    fc6 = tf.nn.relu(tf.matmul(fc5, weights['W_fc6'])+biases['b_fc6'])
    fc6 = tf.nn.dropout(fc6, rate=1-prob)
    print(fc6)
    
    fc7 = tf.nn.relu(tf.matmul(fc6, weights['W_fc7'])+biases['b_fc7'])
    fc7 = tf.nn.dropout(fc7, rate=1-prob)
    print(fc7)
    
    fc8 = tf.nn.relu(tf.matmul(fc7, weights['W_fc8'])+biases['b_fc8'])
    fc8 = tf.nn.dropout(fc8, rate=1-prob)
    print(fc8)
    
    fc9 = tf.nn.relu(tf.matmul(fc8, weights['W_fc9'])+biases['b_fc9'])
    fc9 = tf.nn.dropout(fc9, rate=0)
    print(fc9)
    
    output = tf.matmul(fc9, weights['out'])+biases['out']
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=output))
    optimizer = tf.train.AdamOptimizer(learn_rate).minimize(cost)
    
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(output,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    
    return dict(
              x=x,
              xr = xr,
              xg = xg,
              xb = xb,
              xgs = xgs,
              xsobx = xsobx,
              xsoby = xsoby,
              xlap = xlap,
              xkir = xkir,
              xoptx = xoptx,
              xopty = xopty,
              y=y,
              prob = prob,
              output=output,
              cost=cost,
              optimizer=optimizer,
              accuracy = accuracy
              )

In [0]:
DNN=convolutional_neural_network(keep_rate = 0.5)


filtro
Tensor("Relu:0", shape=(?, 92, 48, 48, 6), dtype=float32)
Tensor("MaxPool3D:0", shape=(?, 46, 24, 24, 6), dtype=float32)
Tensor("Relu_1:0", shape=(?, 40, 22, 22, 16), dtype=float32)
Tensor("MaxPool3D_1:0", shape=(?, 20, 11, 11, 16), dtype=float32)
Tensor("Relu_2:0", shape=(?, 14, 9, 9, 32), dtype=float32)
Tensor("MaxPool3D_2:0", shape=(?, 7, 3, 3, 32), dtype=float32)
Tensor("Relu_3:0", shape=(?, 1, 1, 1, 128), dtype=float32)
filtro
Tensor("Relu_4:0", shape=(?, 92, 48, 48, 6), dtype=float32)
Tensor("MaxPool3D_3:0", shape=(?, 46, 24, 24, 6), dtype=float32)
Tensor("Relu_5:0", shape=(?, 40, 22, 22, 16), dtype=float32)
Tensor("MaxPool3D_4:0", shape=(?, 20, 11, 11, 16), dtype=float32)
Tensor("Relu_6:0", shape=(?, 14, 9, 9, 32), dtype=float32)
Tensor("MaxPool3D_5:0", shape=(?, 7, 3, 3, 32), dtype=float32)
Tensor("Relu_7:0", shape=(?, 1, 1, 1, 128), dtype=float32)
filtro
Tensor("Relu_8:0", shape=(?, 92, 48, 48, 6), dtype=float32)
Tensor("MaxPool3D_6:0", shape=(?, 46, 24, 24, 6), dtype=f

In [0]:
def train_neural_network(DNN, hm_epochs = 10,batch_size=70):
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        #saver.restore(sess,save_file)
        sess.run(tf.global_variables_initializer())
  
        DV = 1
        bestWeights = 0
         ########################################################################## training
        
        for epoch in range(hm_epochs):        
          
            epoch_loss = 0
            epoch_lossVal = 0
            i=0
            acctrain = 0
            contador = 0
            while i < len(dataTrainY)/batch_size:
                start = i
                end = i+batch_size

                feed_dict={DNN["xgs"]:   dataTrainGS[start:end],
                           DNN["xsobx"]: dataTrainSX[start:end],
                           DNN["xsoby"]: dataTrainSY[start:end],
                           DNN["y"]:    dataTrainY[start:end],
                           DNN["prob"]: DV} #antes era 45
                _, c, accuracy  = sess.run([DNN["optimizer"], DNN["cost"]    # _, c, output
                                                 , DNN["accuracy"]], #DNN["output"]
                                                feed_dict=feed_dict)
                acctrain = acctrain + accuracy
                
                epoch_loss += c
                
                contador = contador + 1
                i+=batch_size 
            
             ########################################################################## Validation
            
            
            
            feed_dict={DNN["xgs"]:   dataTrainGS2,
                       DNN["xsobx"]: dataTrainSX2,
                       DNN["xsoby"]: dataTrainSY2,
                       DNN["y"]:     dataTrainY2,
                       DNN["prob"]: 1} #antes era 45
            
            accuracyVal,cVal  = sess.run([DNN["accuracy"],DNN["cost"]],feed_dict=feed_dict)
            epoch_lossVal += cVal
                
            #print('\n\nEpoch', epoch, 'completed out of',hm_epochs)
            #print('LossTraining:\t',epoch_loss,'lossValidation:\t',epoch_lossVal)
            #print('Train: \t  accuracyValidation:\t',accuracyVal, '\t accuracyTraining:\t',acctrain/contador)
            
            if((acctrain/contador) > 0.6 and (acctrain/contador) < 0.8):
              DV = 0.9
            elif((acctrain/contador) > 0.8):
              DV = 0.8
            else:
              DV = 1
            
            ########################################################################## impresion tensorboard training validation
            
            
            tbc.save_value("LossTrain", "lossTrain", epoch, epoch_loss)
            tbc.save_value("AccuracyTrain", "accuracyTrain", epoch, acctrain/contador)
            
            tbc.save_value("LossTrain", "lossValidation", epoch, epoch_lossVal)
            tbc.save_value("AccuracyTrain", "accuracyValidation", epoch, accuracyVal)
            
            tbc.flush_line("lossTrain")
            tbc.flush_line("accuracyTrain")
            tbc.flush_line("lossValidation")
            tbc.flush_line("accuracyValidation")
     
            ########################################################################## test uno (personas que nunca se vieron)
            
            if((epoch+1)%1 == 0):
              
                ta1 = 0 #variable de impresion
                ta11 = 0  #variable de impresion
                
                
                acctest1 = 0
                
                accumaccuracy = 0
                epoch_lossTest = 0
                contadorTotal = 0
                contadorTotalcorrecto = 0
                
                contadorVerdadPositiva = 0
                contadorVerdadNegativa = 0
                contadorMentiraPositiva = 0
                contadorMentiraNegativa = 0
                for x in range(len(dataTestY)):
                  if(len(dataTestGS[x]) != 0 ):
                    feed_dict={DNN["xgs"]:   dataTestGS[x],
                               DNN["xsobx"]: dataTestSX[x],
                               DNN["xsoby"]: dataTestSY[x],
                               DNN["y"]:     dataTestY[x],
                               DNN["prob"]: 1} #antes era 45
                    predictionTest,yTest,accuracyTest,cTest   = sess.run([DNN["output"], DNN["y"], DNN["accuracy"],DNN["cost"]],
                                                                         feed_dict=feed_dict)
                    
                    epoch_lossTest += cTest
                    
                    
                    
                    contadorCorrecto = 0
                    contadorIncorrecto = 0
                    
                    for i in range(len(dataTestY[x])):
                        if((predictionTest[i][0]>predictionTest[i][1] and yTest[i][0]>yTest[i][1]) or
                           (predictionTest[i][0]<predictionTest[i][1] and yTest[i][0]<yTest[i][1])):
                          
                          contadorCorrecto = contadorCorrecto + 1
                          
                          if(x > 20):
                            
                            if((x+1) % 2 == 0):
                              contadorMentiraPositiva += 1
                            else:
                              contadorMentiraNegativa += 1
                              
                          else:
                            
                            if((x+1) % 2 == 0):
                              contadorVerdadPositiva += 1
                            else:
                              contadorVerdadNegativa += 1
                              
                        else:
                          contadorIncorrecto = contadorIncorrecto + 1
                          
                        
                        
                    if(contadorCorrecto >  contadorIncorrecto):
                      accumaccuracy = accumaccuracy + 1
                    contadorTotal = contadorTotal + contadorCorrecto + contadorIncorrecto
                    contadorTotalcorrecto = contadorTotalcorrecto + contadorCorrecto
                    
                acctest1 = contadorTotalcorrecto/contadorTotal
                
                #print('Test1: \t epoch_lossTest1 \t',epoch_lossTest)
                #print('Test1: \t accuracyPorVideoTest1 \t',accumaccuracy/len(dataTestY),'\t accuracyGlobal \t',acctest1)
                #print('Test1: \t contadorMentiraPositiva \t',(contadorMentiraPositiva/contadorTotal)/acctest1,'\t contadorMentiraNegativa \t',(contadorMentiraNegativa/contadorTotal)/acctest1)
                #print('Test1: \t contadorVerdadPositiva \t',(contadorVerdadPositiva/contadorTotal)/acctest1,'\t contadorVerdadNegativa \t',(contadorVerdadNegativa/contadorTotal)/acctest1)
                
                
                
                
                ########################################################################## impresion tensorboard test1
                tbc.save_value("AccuracyTest", "accuracyGlobal1", epoch, acctest1)
                tbc.flush_line("accuracyGlobal1")
                
                tbc.save_value("LossTest", "lossTest", epoch, epoch_lossTest)
                tbc.save_value("AccuracyTest", "accuracyPorVideo1", epoch, accumaccuracy/len(dataTestY))
                tbc.flush_line("lossTest")
                
                ######################################################################### Test dos (personas ya vistas) verdad
                
                
                ta2 = 0  #variable de impresion
                ta22 = 0  #variable de impresion
                
                
                acctest2 = 0   
                   
                accumaccuracy = 0
                epoch_lossTest = 0
                contadorTotal = 0
                contadorTotalcorrecto = 0
                   
                   
                contadorVerdadNegativa = 0
                contadorMentiraPositiva = 0
                   
                acctotal = 0
                   
                for x in range(len(dataTestTruthY2)+len(dataTestDeceitY2)):
                  if((x+1) < len(dataTestTruthY2)):
                    feed_dict={DNN["xgs"]:   dataTestTruthGS2[x],
                               DNN["xsobx"]: dataTestTruthSX2[x],
                               DNN["xsoby"]: dataTestTruthSY2[x],
                               DNN["y"]:     dataTestTruthY2[x],
                               DNN["prob"]: 1} 
                    predictionTest,yTest,accuracyTest,cTest   = sess.run([DNN["output"], DNN["y"], DNN["accuracy"],DNN["cost"]],
                                                                         feed_dict=feed_dict)
                    epoch_lossTest += cTest
                    
                    contadorCorrecto = 0
                    contadorIncorrecto = 0
                   
                    for i in range(len(dataTestTruthY2[x])):
                        if((predictionTest[i][0]>predictionTest[i][1] and yTest[i][0]>yTest[i][1]) or
                           (predictionTest[i][0]<predictionTest[i][1] and yTest[i][0]<yTest[i][1])):
                          contadorCorrecto = contadorCorrecto + 1
                        else:
                          contadorIncorrecto = contadorIncorrecto + 1
                    if(contadorCorrecto >  contadorIncorrecto):
                      accumaccuracy = accumaccuracy + 1
                      contadorVerdadNegativa += 1
                   
                    contadorTotal = contadorTotal + contadorCorrecto + contadorIncorrecto
                    contadorTotalcorrecto = contadorTotalcorrecto + contadorCorrecto
                    
                  else:
                    feed_dict={DNN["xgs"]:   dataTestDeceitGS2[x-len(dataTestTruthY2)],
                               DNN["xsobx"]: dataTestDeceitSX2[x-len(dataTestTruthY2)],
                               DNN["xsoby"]: dataTestDeceitSY2[x-len(dataTestTruthY2)],
                               DNN["y"]:     dataTestDeceitY2[x-len(dataTestTruthY2)],
                               DNN["prob"]: 1} 
                    predictionTest,yTest,accuracyTest,cTest   = sess.run([DNN["output"], DNN["y"], DNN["accuracy"],DNN["cost"]],
                                                                         feed_dict=feed_dict)
                    
                    epoch_lossTest += cTest
                    
                    contadorCorrecto = 0
                    contadorIncorrecto = 0
                   
                    for i in range(len(dataTestDeceitY2[x-len(dataTestTruthY2)])):

                        if((predictionTest[i][0]>predictionTest[i][1] and yTest[i][0]>yTest[i][1]) or
                           (predictionTest[i][0]<predictionTest[i][1] and yTest[i][0]<yTest[i][1])):
                          contadorCorrecto = contadorCorrecto + 1

                        else:
                          contadorIncorrecto = contadorIncorrecto + 1
                    if(contadorCorrecto >  contadorIncorrecto):
                      accumaccuracy = accumaccuracy + 1
                      contadorMentiraPositiva += 1
                    contadorTotal = contadorTotal + contadorCorrecto + contadorIncorrecto
                    contadorTotalcorrecto = contadorTotalcorrecto + contadorCorrecto
                    
                #print('Test2: \t epoch_lossTest2 \t',epoch_lossTest)
                #print('Test2: \t accuracyPorVideo2 \t',accumaccuracy/(len(dataTestDeceitY2)+len(dataTestTruthY2)),'\t accuracyGlobal2 \t',contadorTotalcorrecto/contadorTotal)
                #print('Test2: \t contadorMentiraPositiva \t',contadorMentiraPositiva/len(dataTestDeceitY2),'\t contadorVerdadNegativa \t',contadorVerdadNegativa/len(dataTestTruthY2))
                
                
                
                ########################################################################## impresion tensorboard test2
                
                tbc.save_value("AccuracyTest", "accuracyGlobal2", epoch, contadorTotalcorrecto/contadorTotal)
                tbc.flush_line("accuracyGlobal2")
                
                tbc.save_value("LossTest", "lossTest2", epoch, epoch_lossTest)
                tbc.save_value("AccuracyTest", "accuracyPorVideo2", epoch, accumaccuracy/(len(dataTestTruthY2)+len(dataTestDeceitY2)))
                tbc.flush_line("lossTest2")
                
                roar = contadorTotalcorrecto/contadorTotal
                
                ########################################################################## condicion para guardar los pesos de la red
                
                if(roar > 0.60 and roar > bestWeights and acctrain/contador > 0.60):
                  saver.save(sess,save_file)
                  bestWeights = roar
                  print('\n\nEpoch', epoch, 'completed out of',hm_epochs)
                  print('LossTraining:\t',epoch_loss,'lossValidation:\t',epoch_lossVal)
                  print('Train: \t  accuracyValidation:\t',accuracyVal, '\t accuracyTraining:\t',acctrain/contador)
                  print('Test2: \t epoch_lossTest2 \t',epoch_lossTest)
                  print('Test2: \t accuracyPorVideo2 \t',accumaccuracy/(len(dataTestDeceitY2)+len(dataTestTruthY2)),'\t accuracyGlobal2 \t',contadorTotalcorrecto/contadorTotal)
                  print('Test2: \t contadorMentiraPositiva \t',contadorMentiraPositiva/len(dataTestDeceitY2),'\t contadorVerdadNegativa \t',contadorVerdadNegativa/len(dataTestTruthY2))

        tbc.close()



In [0]:
tbc=TensorBoardColab()



Wait for 8 seconds...
TensorBoard link:
https://bbb4c63d.ngrok.io


In [0]:
train_neural_network(DNN,hm_epochs = 4000,batch_size=150)
